In [1]:
import re
import pandas as pd
from itertools import product

In [2]:
with open('day-14-input.txt', 'r') as f:
    lines = [l.rstrip() for l in f.readlines()]

In [3]:
def int2bytes(value):
    v = value
    l = []
    for i in range(36):
        l.append(v % 2)
        v = v >> 1
    return list(reversed(l))

def bytes2int(bs):
    n = 0
    for i, b in enumerate(reversed(bs)):
        n += b * (2 ** i)
    return n

# Part 1

In [4]:
def masked(value, mask):
    bs = int2bytes(value)
    res = mask.copy()
    for i, m in enumerate(mask):
        if mask[i] is None:
            res[i] = bs[i]
    return bytes2int(res)

In [5]:
mask = [None] * 36
mem = {}
for inst in lines:
    if m := re.match('mask = ([X10]{36})', inst):
        mask = [int(b) if b != 'X' else None for b in list(m.group(1))]
    elif m := re.match('mem\[([0-9]+)\] = ([0-9]+)', inst):
        addr = int(m.group(1))
        value = int(m.group(2))
        mem[addr] = masked(value, mask)

n = 0
for v in mem.values():
    n += v

print(f'Part 1: {n}')

Part 1: 6386593869035


# Part 2

In [6]:
def write_to_memory(mem, addr, mask, value):
    res = int2bytes(addr)
    for i, b in enumerate(mask):
        if b is None:
            res[i] = None
        elif b == 1:
            res[i] = 1
    
    nones = [i for i, b in enumerate(res) if b is None]
    s = pd.Series(res)
    for t in product([0, 1], repeat=len(nones)):
        s.loc[nones] = t
        mem[bytes2int(s.values)] = value
    

In [7]:
mask = [None] * 36
mem = {}

for inst in lines:
    if m := re.match('mask = ([X10]{36})', inst):
        mask = [int(b) if b != 'X' else None for b in list(m.group(1))]
    elif m := re.match('mem\[([0-9]+)\] = ([0-9]+)', inst):
        addr = int(m.group(1))
        value = int(m.group(2))
        write_to_memory(mem, addr, mask, value)

n = 0
for v in mem.values():
    n += v

print(f'Part 2: {n}')

Part 2: 4288986482164
